## <center>Лабораторная работа № 6 'Прогноз успеха фильмов по обзорам'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Прогноз успеха фильмов по обзорам (Predict Sentiment From Movie Reviews)

### Задачи
* Ознакомиться с задачей классификации
* Изучить способы представления текста для передачи в ИНС
* Достигнуть точность прогноза не менее 95%

### Требования
1. Построить и обучить нейронную сеть для обработки текста
2. Исследовать результаты при различном размере вектора представления текста
3. Написать функцию, которая позволяет ввести пользовательский текст (в отчете 
привести пример работы сети на пользовательском тексте

In [1]:
import numpy as np
import torch
import re
import spacy

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D
from keras.datasets import imdb
from keras.regularizers import l2

import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform

import gensim
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [4]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

Categories: [0 1]
Number of unique words: 9998


In [5]:
length = [len(i) for i in data]

In [6]:
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173


In [7]:
print("Label:", targets[0])
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [9]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

data = vectorize(data)
targets = np.array(targets).astype("float32")

In [10]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [11]:
model = Sequential()

model.add(Dense(50, activation = "relu", input_shape=(10000, )))

model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))

model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))

model.add(Dense(1, activation = "sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500050    
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-trai

In [12]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", 
              metrics = ["accuracy"])

In [13]:
results = model.fit(train_x, train_y, epochs= 2, batch_size = 500, 
                    validation_data = (test_x, test_y))

Epoch 1/2
80/80 [==============================] - 3s 25ms/step - loss: 0.4213 - accuracy: 0.8106 - val_loss: 0.2640 - val_accuracy: 0.8933
Epoch 2/2
80/80 [==============================] - 2s 20ms/step - loss: 0.2211 - accuracy: 0.9160 - val_loss: 0.2585 - val_accuracy: 0.8955


In [14]:
print(np.mean(results.history["val_accuracy"]))

0.8944000005722046


### Достигнем максимально возможной точности прогноза на валидационных данных (без применения Transfer Learning, Ensemble Modeling, Word Embeddings, etc)

In [15]:
model_exp = Sequential()

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(60, activation = "relu", input_shape=(10000, ),
                       kernel_initializer=glorot_uniform()))
model_exp.add(Dense(150, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dropout(0.3))

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(90, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dropout(0.2))

model_exp.add(BatchNormalization(synchronized=True))

model_exp.add(Dense(40, activation='relu',
                   kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model_exp.add(Dense(1, activation='sigmoid'))

In [16]:
model_exp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_exp.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=350, verbose=1)
print(np.mean(results.history["val_accuracy"]))

Epoch 1/3
115/115 [==============================] - 7s 40ms/step - loss: 0.4059 - accuracy: 0.8076 - val_loss: 0.3257 - val_accuracy: 0.8761
Epoch 2/3
115/115 [==============================] - 4s 37ms/step - loss: 0.1794 - accuracy: 0.9323 - val_loss: 0.2770 - val_accuracy: 0.8847
Epoch 3/3
115/115 [==============================] - 4s 37ms/step - loss: 0.1024 - accuracy: 0.9627 - val_loss: 0.3099 - val_accuracy: 0.8847
0.8944000005722046


### Функция, которая позволяет ввести пользовательский текст

In [17]:
def preproc(text):
    text_punct = re.sub(r"[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]", ' ', text)
    text_numbers = re.sub(r"[0-9]", ' ', text_punct)
    
    text_token = word_tokenize(text_numbers, language='english')
#     stop = set(stopwords.words('english'))
    text_preproc = [word for word in text_token]
    
    return text_preproc

In [18]:
def get_index(words_prep):
    res = []
    for i in range(len(words_prep)):
        word_index = imdb.get_word_index()
        word = words_prep[i]
        if word_index[word] < 10000:
            res.append(word_index[word])
    return res

In [19]:
def get_res(pred):
    if pred>0.5:
        print(f'Great film I guess {pred}') 
    else: print(f'What was all the puff about that... {pred}')

#### Positive review

In [20]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
words_idx = get_index(words)

Input your movie review: When you very nearly spray a mouthful of drink over the person in front of you, its generally a good indicator the movie is pretty funny. The sandwich joke had me in stitches! This movie doesn't rely on just a few jokes to carry it, they maintain a subtle layer of humour throughout and then have you in stitches with some brilliant jokes. The cast in this movie are well picked and really gets the whole dilemma of everyday life as a vampire! Hopefully this gets picked up and the masses get a chance to enjoy this wee gem. Loved this movie and would definitely recommend it. Gave this a 10. A must see!!


In [21]:
fin_words = vectorize([words_idx])

In [22]:
pred = model.predict(fin_words)[0][0]

1/1 [==============================] - 0s 123ms/step


In [23]:
get_res(pred)

Great film I guess 0.863459050655365


#### Negative review

In [24]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
words_idx = get_index(words)

Input your movie review: It's a refreshing idea but this is amateur hour and looks like a film made by a bunch of film students. I didn't laugh once, not even a giggle. 95% of it is filmed inside a dark house which just adds to the dreariness of it all. It's slow and I can't see it going into cinema release in the states. I have liked a few oddball comedies from New Zeland but I will remember this one for all the wrong reasons. Just awful!


In [25]:
fin_words = vectorize([words_idx])

In [26]:
pred = model.predict(fin_words)[0][0]

1/1 [==============================] - 0s 19ms/step


In [27]:
get_res(pred)

What was all the puff about that... 0.3691744804382324
